## Notebook to prototype Chip Counting Algorithm
Given image, output the number of chips present

Potential Problems:
* Overlapping Chips
* Different Coloured Chips

In [1]:
import cv2 as cv
import os
import numpy as np
import pandas as pd
from helper_functions import read_imgs

In [2]:
img_list = read_imgs(r"C:\Users\Asus\Downloads\nus-sds-dsc2021\train_data\train_images")

In [162]:
df = pd.read_csv(r"C:\Users\Asus\Downloads\nus-sds-dsc2021\train_data\train_labels.csv")
df

,img_id,object_count_gt,handwritten_count_gt,anomalies_bbox_gt
0,1001,80,80,[]
1,1002,6,6,[]
2,1003,51,52,"[('black_spot', [67, 44, 27, 16])]"
3,1004,24,24,[]
4,1005,77,77,[]
...,...,...,...,...
111,1112,200,-1,[]
112,1113,200,-1,[]
113,1114,198,-1,[]
114,1115,200,-1,[]


In [171]:
pred = []
for i in range(len(img_list)):
    count = 0
    img = cv.resize(img_list[i].copy(), (round(440*0.9), round(610*0.9)))
    lab= cv.cvtColor(img, cv.COLOR_BGR2LAB)
    l, a, b = cv.split(lab)

    ret, th = cv.threshold(b, 145, 255, cv.THRESH_BINARY);
#     ret, th = cv.threshold(b, 127, 255, cv.THRESH_OTSU);
    
    contours, hierarchy = cv.findContours(th, mode=cv.RETR_TREE, method=cv.CHAIN_APPROX_SIMPLE)
    mask = np.zeros_like(img)
    for cnt in contours:
        approx = cv.approxPolyDP(cnt, 0.01 * cv.arcLength(cnt, True), True)
        area = cv.contourArea(approx)
        if 150 < area:
            cv.drawContours(mask, [approx], -1, (0, 255, 0), 1)
            count += area//100
        elif 50 < area < 150:
            cv.drawContours(mask, [approx], -1, (0, 255, 0), 1)
            count += 1
        else:
            cv.drawContours(mask, [approx], -1, (255, 0, 0), 1)
    pred.append(int(count))


#     th = cv.cvtColor(th, cv.COLOR_GRAY2BGR)
#     b = cv.cvtColor(b, cv.COLOR_GRAY2BGR)
    
#     cv.imshow("window", cv.hconcat([th, img, b]))
#     cv.waitKey(0)
#     cv.destroyAllWindows()
    

In [193]:
error = 0
for i in range(len(pred)):
    error += abs(pred[i] - df["object_count_gt"][i])
    
print(error)

1998


In [201]:
# 32 for merge issue
img = cv.resize(img_list[31].copy(), (round(440*0.9), round(610*0.9)))
count = 0

lab= cv.cvtColor(img, cv.COLOR_BGR2LAB)
l, a, b = cv.split(lab)

ret, th = cv.threshold(b, 145, 255, cv.THRESH_BINARY);
# ret, th = cv.threshold(b, 0, 255, cv.THRESH_OTSU);

contours, hierarchy = cv.findContours(th, mode=cv.RETR_TREE, method=cv.CHAIN_APPROX_SIMPLE)
mask = np.zeros_like(img)
# x = cv.drawContours(mask, contours, -1, (0, 255, 0), 1)
for cnt in contours:
    approx = cv.approxPolyDP(cnt, 0.01 * cv.arcLength(cnt, True), True)
    area = cv.contourArea(approx)
    if 150 < area:
        cv.drawContours(mask, [approx], -1, (0, 255, 0), 1)
        count += area//100
        print(area, area//100)
    elif 50 < area < 150:
        cv.drawContours(mask, [approx], -1, (0, 0, 255), 1)
        count += 1
    else:
        cv.drawContours(mask, [approx], -1, (255, 0, 0), 1)

print(count)

266.5 2.0
33.0


In [202]:
th = cv.cvtColor(th, cv.COLOR_GRAY2BGR)
b = cv.cvtColor(b, cv.COLOR_GRAY2BGR)
cv.imshow("window", cv.hconcat([th, img, b]))
# cv.imshow("window", mask)
cv.waitKey(0)
cv.destroyAllWindows()

In [ ]:
# img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
# blur = cv.bilateralFilter(img,30,40,40)

# th = cv.adaptiveThreshold(blur, maxValue=255, blockSize=11, adaptiveMethod=cv.ADAPTIVE_THRESH_GAUSSIAN_C, thresholdType=cv.THRESH_BINARY, C=2)
# img_morph = cv.morphologyEx(th, cv.MORPH_CLOSE, (5,5), iterations=1)

# # dst = img_morph

# contours, hierarchy = cv.findContours(img_morph, mode=cv.RETR_TREE, method=cv.CHAIN_APPROX_SIMPLE)
# mask = np.zeros_like(img_list[0].copy())
# # cv.drawContours(mask, contours, -1, (0, 255, 0), 1)
# for cnt in contours:
#     approx = cv.approxPolyDP(cnt, 0.01 * cv.arcLength(cnt, True), True)
#     if (len(approx) != 4):
#         continue
#     cv.drawContours(mask, [approx], -1, (0, 255, 0), 1)

# dst = cv.Canny(blur, threshold1=5, threshold2=50, edges=None, apertureSize=5)
# lines = cv.HoughLines(dst, rho=1, theta=np.pi/180, threshold=150, lines=None, srn=0, stn=0,)# min_theta=0, max_theta=np.pi/2+0.2)
# lines = cv.HoughLinesP(dst, rho=1, theta=np.pi/180, threshold=50, lines=None, minLineLength=100, maxLineGap=100)

# if lines is not None:
#     for i in range(0, len(lines)):
#         line = lines[i][0]
#         cv.line(img, (line[0], line[1]), (line[2], line[3]), (0,0,255),1)

# for rho,theta in lines[0]:
#     a = np.cos(theta)
#     b = np.sin(theta)
#     x0 = a*rho
#     y0 = b*rho
#     x1 = int(x0 + 1000*(-b))
#     y1 = int(y0 + 1000*(a))
#     x2 = int(x0 - 1000*(-b))
#     y2 = int(y0 - 1000*(a))
#     cv.line(img,(x1,y1),(x2,y2),(0,0,255),3)

# len(lines)